In [ ]:
# 12/12/2021-1
# https://www.tensorflow.org/hub/tutorials/tf2_image_retraining
from __future__ import absolute_import
from IPython.display import clear_output
from BashColors import C
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from os.path import exists, join

contentPath = os.getcwd()
cv2Path=join(contentPath, 'CV2Images')
testPath=join(contentPath, 'images')
checkpointPath = join(contentPath, 'CheckPoints')

import itertools
import glob, os, time
from time import sleep

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

for fil in glob.glob('*.h5'):
    fullPath=os.path.abspath(fil)
    print(f'removing: {C.BIRed}{fullPath}{C.ColorOff}')
    os.remove(fullPath)

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print('cwd: ', os.getcwd())

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=1, verbose=1,
    mode='auto', baseline=None, restore_best_weights=False
)

checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

In [ ]:
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 4

import tensorflow
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

validation_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    batch_size = BATCH_SIZE)
# clear_output()

In [ ]:

class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

do_data_augmentation = True

if do_data_augmentation:
    preprocessing_model.add(tf.keras.layers.RandomRotation(20))
    preprocessing_model.add(
        tf.keras.layers.RandomTranslation(0, 0.1))
    preprocessing_model.add(
        tf.keras.layers.RandomTranslation(0.1, 0))
    # Like the old tf.keras.preprocessing.image.ImageDataGenerator(),
    # image sizes are fixed when reading, and then a random zoom is applied.
    # If all training inputs are larger than image_size, one could also use
    # RandomCrop with a batch size of 1 and rebatch later.
    preprocessing_model.add(tf.keras.layers.RandomZoom(-0.1, 0.1))
    preprocessing_model.add(
        tf.keras.layers.RandomFlip(mode="horizontal"))
    
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

In [ ]:
# val_ds = build_dataset("validation")
valid_size = validation_ds.cardinality().numpy()
val_ds = validation_ds.unbatch().batch(BATCH_SIZE)
val_ds = validation_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 4

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential(name='Defcon4_V1')
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

thisHistory = model.fit(train_ds,
                        epochs=10,
                        steps_per_epoch=(173 // 4),
                        validation_steps=(43 // 4),
                        callbacks=[earlyStop, checkpoints])

thisHistory.history

In [ ]:
modelName = model.name + '.h5'
modelSavePath = join(contentPath, modelName)
model.save(modelSavePath, overwrite=True)

In [ ]:
%ls
import cv2
from CV2_Utils import *
newModel = tf.keras.models.load_model(Def, compile=True)
# element=test_ds.get_single_element()
img = test_ds.take(1)
# cvu.plotShowSingleImage(img)
print(type(img))
model.predict()